In [1]:
import keras
import pickle
import numpy as np
import os
from keras.models import Model
from keras.layers import Input, Dense, Reshape, dot
from keras.layers.embeddings import Embedding

Using TensorFlow backend.
c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
names=["vocab_size","labels","word_target","word_context","window_size"]
for name in names:
    with open(os.path.join(os.path.join(os.path.abspath('../binaries'),"pickles"),name+".pickle"), "rb") as f:
        globals()[name]=pickle.load(f)

In [3]:
vector_dim=300
epochs=200000

In [4]:
input_target = Input((1,))
input_context = Input((1,))

W1014 18:57:48.717042 18820 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1014 18:57:48.744995 18820 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [5]:
embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')
target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

W1014 18:57:48.845725 18820 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [6]:
# setup a cosine similarity operation which will be output in a secondary model
similarity = dot([target,context],axes=1,normalize=True)

In [7]:
# now perform the dot product operation to get a similarity measure
dot_product = dot([target, context], normalize=False, axes=1)
dot_product = Reshape((1,))(dot_product)
# add the sigmoid output layer
output = Dense(1, activation='sigmoid')(dot_product)
# create the primary training model
model = Model(input=[input_target, input_context], output=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  import sys
W1014 18:57:48.988342 18820 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1014 18:57:48.999313 18820 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W1014 18:57:49.003303 18820 deprecation.py:323] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future ver

In [8]:
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 1000 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))

W1014 18:57:49.174255 18820 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Iteration 0, loss=0.6949827671051025
Iteration 1000, loss=0.6922613978385925
Iteration 2000, loss=0.687083899974823
Iteration 3000, loss=0.6899444460868835
Iteration 4000, loss=0.6994915008544922
Iteration 5000, loss=0.7002225518226624
Iteration 6000, loss=0.6933541893959045
Iteration 7000, loss=0.7081913948059082
Iteration 8000, loss=0.7037034034729004
Iteration 9000, loss=0.6865083575248718
Iteration 10000, loss=0.692814290523529
Iteration 11000, loss=0.6690691709518433
Iteration 12000, loss=0.6581525802612305
Iteration 13000, loss=0.6827358603477478
Iteration 14000, loss=0.6958131194114685
Iteration 15000, loss=0.7059758901596069
Iteration 16000, loss=0.6846375465393066
Iteration 17000, loss=0.6785730123519897
Iteration 18000, loss=0.6893879771232605
Iteration 19000, loss=0.7080847024917603
Iteration 20000, loss=0.6864486336708069
Iteration 21000, loss=0.6967442035675049
Iteration 22000, loss=0.726808488368988
Iteration 23000, loss=0.6900349259376526
Iteration 24000, loss=0.67908185

Iteration 198000, loss=0.5163780450820923
Iteration 199000, loss=0.9668267369270325


In [9]:
model.save(os.path.join(os.path.join(os.path.abspath('../binaries'),"models"),"classification_model.h5"))